## Fig 2b

In [ ]:
library(tibble)
library(enrichplot)
library(RColorBrewer)
library(ggplot2)
library(ggpubr)
library(dplyr)
library(tidyverse)
library(pheatmap)
library(ComplexHeatmap)
library(circlize)
library(scales)

In [ ]:
df <- read.csv("./compartment_composition_in_samples.csv", row.names=1, check.names=FALSE)


In [ ]:
mat <- as.matrix(df[, setdiff(colnames(df), c("Tumor_Type", "cluster", "immune_state"))])
mat_scaled <- scale(mat)

meta_info <- df[, c("Tumor_Type", "cluster", "immune_state")]
meta_info$Sample <- rownames(df)


ht <- Heatmap(
  mat_scaled,
  cluster_rows = FALSE,
  cluster_columns = TRUE,
  col = colorRamp2(c(-2, 0, 4), c("blue", "white", "red")),
  row_labels = row_labels,
  row_names_gp = gpar(fontsize = 6),
  row_names_side = "right"
)


In [ ]:
row_ordered_index <- row_order(ht)
ordered_row_names <- rownames(mat_scaled)[row_ordered_index]


In [ ]:
clinical_data <- read.csv("./HD100_clinical_info.csv",row.names=1)
clinical_data <- clinical_data[ordered_row_names,]
clinical_data <- clinical_data[,c("Tumor_Type","Tumor_SubType","Age","Recurrence_Status","Tumor_Grade","Tumor_Stage","MSI_state","CIN_score","Neoadjuvant_Treatment","cluster")]

clinical_data[is.na(clinical_data)] <- "Unknown"
clinical_data[clinical_data==""] <- "Unknown"

age_color_fun <- colorRamp2(
  breaks = c(min(clinical_data$Age), max(clinical_data$Age)),
  colors = c("#ffdbe0", "#e32636")
)

CIN_color_fun <- colorRamp2(
  breaks = c(min(clinical_data$CIN_score), max(clinical_data$CIN_score)),
  colors = c("#fee2d5", "#cc5500")
)

top_annotations <- HeatmapAnnotation(
  df = clinical_data,
  col = list(
    Tumor_Type = c("OC" = "#ab526a", "EC" = "#d68081", "CC"="#2978b5", 'Unknown' = "#dcdcdc"),
    Tumor_SubType = c("Mucinous" = "#b1bcf4", "Serous" = "#65a56d", "Clear cell" = "#f4e88d", "Squ"="#f85adb", "Ade"="#a94684", "Ade-Squ"="#ff1e16", "Endometrial" = "#eab768", "Sarcoma" = "#cf5b5d",'Others' = "#dcdcdc"),
    Age = age_color_fun,
    Recurrence_Status = c("1" = "#785190", "0" = "#6f87bb", 'Unknown' = "#dcdcdc"),
    Tumor_Grade = c("1" = "#a1a5bf", "2" = "#656e96", "3" = "#475482", 'Unknown' = "#dcdcdc"),
    Tumor_Stage = c("I" = "#a6c692", "II" = "#92b17f", "III" = "#7e9d6b", "IV" = "#6b8959", 'Unknown' = "#dcdcdc"),
    MSI_state = c("MSI-low" = "#62b58f", "MSI-high" = "#e7cc5e", 'Unknown' = "#dcdcdc"),
    CIN_score = CIN_color_fun,
    Neoadjuvant_Treatment = c("Pre-treatment" = "#20B2AA", "Chemotherapy" = "#E3C576", "Chemotherapy/Immune" = "#E5A76C", "Chemotherapy/Targeted" = "#D87558", 'Unknown' = "#dcdcdc"),
    cluster = c("1" = "#b1bcf4", "2" = "#65a56d", "3" = "#eab768", '4' = "#f4e88c", '5' = "#e73828", '6' = "#f6becd")
  ),
  annotation_name_side = "left",
  annotation_legend_param = list(
    Tumor_Type = list(title = "Purity", title_gp = gpar(fontsize = 13)),
    Tumor_SubType = list(title = "Tumor_Type", title_gp = gpar(fontsize = 13)),
    Age = list(
      title = "Age", 
      title_gp = gpar(fontsize = 13),
      color_bar = "continuous"
    ),
    Recurrence_Status = list(title = "Recurrence_Status", title_gp = gpar(fontsize = 13)),
    Tumor_Grade = list(title = "Tumor_Grade", title_gp = gpar(fontsize = 13)),
    Tumor_Stage = list(title = "Tumor_Stage", title_gp = gpar(fontsize = 13)),
    MSI_state = list(title = "MSI_state", title_gp = gpar(fontsize = 13)),
    CIN_score = list(
      title = "CIN_score", 
      title_gp = gpar(fontsize = 13),
      color_bar = "continuous"
    ),
    Neoadjuvant_Treatment = list(title = "Neoadjuvant_Treatment", title_gp = gpar(fontsize = 13))
)


In [ ]:
heatmap_with_annotations <- Heatmap(matrix(nrow = 0, ncol = 100),  # 空矩阵
                                    top_annotation = top_annotations,
                                    show_row_names = FALSE,
                                    show_column_names = FALSE)

annotation <- draw(heatmap_with_annotations, annotation_legend_side = "right")
